### Is my microphone going to work with my sound card at a certain SPL?
Let's start by importing this NumPy library otherwise we cannot make the calculations.

In [1]:
import numpy as np

Let's input the microphone's sensitivity. Sensitivity of the microphone, which is a device that generally converts sound pressure into electrical voltage, is a quality, that sound describe what the output (voltage) of the device is when exposed some input (sound pressure). 

The sensitivity is expressed in mV/Pa, which expresses that voltage ('how many milivolts') we get out of the microphone when we expose it to 1 pascal of acoustic pressure. 

In [2]:
micSens = 15 #in mV/Pa

Just a short reminder of what SPL (sound pressure level) is:

In [3]:
def SPL(x):
    return 20*np.log10(x/0.00002) # The reference sound pressure is 20 micropascals
print(f'1 Pa = {SPL(1):.1f} dB SPL')

1 Pa = 94.0 dB SPL


Let's calcute the other way to define microphone sensitivity. This other way is to take this voltage you get out of the microphone at 94 dB SPL and express it in respect with 1 V. This shows how many times smaller (or bigger) the output of this microphone is compared to 1 volt. Or: How many times quiter this microphone's output is when compared to a microphone that could produce 1 V when exposed to 1 Pa (or 94 dB SPL).

When you take a voltage ratio of x volts over 1 volt, and you express it in decibels you literally get the unit of dBV, because the **V** in dB**V**,  stands for 1 V as the reference voltage. 

In [4]:
micSens_dBV = 20*np.log10(micSens/1000)
print(f'The microphone sensitivity is {micSens:.1f}'
      f' mV/Pa \n which is the same'
      f' as {micSens_dBV:.1f} dBV or as some \n manufacturers write it:'
      f' {micSens_dBV:.1f} dB (re 1V/Pa).')

The microphone sensitivity is 15.0 mV/Pa 
 which is the same as -36.5 dBV or as some 
 manufacturers write it: -36.5 dB (re 1V/Pa).


Now we need to get to **dBu** values at some devices. The **dBu** and **dBV** units are almost the same thing, they both express voltage ratios, but **dBu** doesn't use 1V as a reference but a slighly different voltage of 

In [5]:
dBu_REF=np.sqrt(0.001*600) #voltage of 1 mW into 600 ohm load
print(f'{dBu_REF:.4f} V')

0.7746 V


It doesn't matter whether the **u** in dB**u** stands for 'volume **u**nit' or '**u**nloaded' or ''**u**nterminated' or anything else. The only important thing is the reference voltage above, that's used in the dBu calculation.


Let's express the voltage of 1V using **dBu** and **dBV** and then do the same thing for 0.7746 V.
First I'll define the **dBu** and **dBV** calculation functions, they might also get useful later.

In [6]:
def dBu(x): #function to calculate dBu from voltage
    return 20*np.log10(x/dBu_REF)
def dBV(x): #function to calculate dBV from voltage
    return 20*np.log10(x/1)

In [7]:
print(f'1 V = {dBu(1):.3f} dBu = {dBV(1):.1f} dBV')
print(f'0.7746 V = {dBu(0.7746):.1f} dBu = {dBV(0.7746):.3f} dBV')

1 V = 2.218 dBu = 0.0 dBV
0.7746 V = 0.0 dBu = -2.218 dBV


So whenever we see dBu and dBV next to each other, there's just about 2 dB difference between those two numbers. 

Anyway, let's express the microphone output voltage at 94 dB of sound pressure level (1 Pa) in dBu:

In [8]:
micSens_dBu=dBu(micSens/1000) #
print(f"The microphone's output at 94 dB SPL is {micSens_dBu:.1f} dBu.")

The microphone's output at 94 dB SPL is -34.3 dBu.


As of this moment, neither the crest factor of the signal nor the measurement integration time have been specified. We know nothing about neither the electric nor the acoustic side of things. And since it's not specified, let's expect that the good people that measured those values used the same arithmetics for both domains. 

Therefore we should be able to input a constant RMS signal (like for example a sine wave) and if we measure the RMS values at the output, it should be just as we calculated. Or when we input a signal with a certain peak, we should measure the peak we voltage as calculated using the math above. So let's try not to confuse what our signals are and how we measure them at each domain.

Sensitivity is just a constant, a simple multiplier. If input something on one side, it gets multiplied and comes out on the other side. 

So let's assume we have an acoustical signal with peaks of 1 Pa. 

If this is a sine wave, the RMS value (that you would see on an sound level meter) is 0.707 Pa (sine wave has a crest factor of 1.4:1 or 3 dB). So the SPL value of the RMS at your sound level meter would be 91 dB. It doesn't matter if you choose Fast (125 ms), Slow (1000 ms) or LEQ of hours, any time weighting it should still read 91 dB SPL. And if the device could actually show peaks, those peaks should be 94 dB SPL.

If this signal we have is a standard pink noise with a crest factor of 4:1 or 12 dB and we still have peaks of 1 Pa, then the RMS value would be 4 times lower or lower by 12 dB, which makes it 0.25 Pa. The RMS that's measured by a sound level meter (Fast/Slow/any LEQ) is 82 dB SPL.

In [9]:
print(f'1 Pa = {SPL(1):.1f} dB SPL')
print(f'0.707 Pa = {SPL(0.707):.1f} dB SPL')
print(f'0.25 Pa = {SPL(0.25):.1f} dB SPL')

1 Pa = 94.0 dB SPL
0.707 Pa = 91.0 dB SPL
0.25 Pa = 81.9 dB SPL


Let's stay with peaks now, because we want to be able to measure peak values of SPL.

Let's specify the maximum SPL (peak) we want our system to measure: 

In [10]:
desiredMaxSPL = 140 #in dB SPL

What's the dB relationship between this value and the reference SPL that was used assess the sensitivity? 

In [11]:
diff_dB=desiredMaxSPL-SPL(1)
print(f'The difference is {diff_dB:.1f} dB')

The difference is 46.0 dB


Let's calculate the ratio that equals to this decibel value. Which is an equvivalent question of: What number to I need to use, if I want to multiply it with he original SPL or the original voltage in order to get to the final SPL or the final voltage?

In [12]:
diff_ratio=10**(diff_dB/20)
print(f'The linear ratio of {diff_dB:.1f} dB is {diff_ratio:.0f}/1.')

The linear ratio of 46.0 dB is 200/1.


So if the microphone is actually capable of converting the desired SPL into voltage, this would be the outcome:

In [13]:
micOut_volts = diff_ratio*micSens/1000
micOut_dBV = dBV(diff_ratio*micSens/1000)
micOut_dBu = dBu(diff_ratio*micSens/1000)
print(f"The microphone's output at {desiredMaxSPL} SPL is {micOut_volts:.2f} V.")
print(f"The microphone's output at {desiredMaxSPL} SPL is {micOut_dBV:.1f} dBV.")
print(f"The microphone's output at {desiredMaxSPL} SPL is {micOut_dBu:.1f} dBu.")

The microphone's output at 140 SPL is 3.00 V.
The microphone's output at 140 SPL is 9.5 dBV.
The microphone's output at 140 SPL is 11.8 dBu.


So now we know what our peak voltage output is produced by the at our desired max SPL. 

### Is it safe to use a certain microphone to measure certain max SPL peaks using OctaCapture inputs?

Let's have a look at the soundcards input capabilities first.

The OctaCapture's manual gives us the following data:

In [14]:
nom_dBu_XLR1_6 = -6 # [dBu] nominal voltage for XLR 1-6, minimum gain
nom_dBu_XLR7_8 = 0 # [dBu] nominal voltage for XLR 7-8, minimum gain
nom_dBu_TRS1_8 = 4 # [dBu] nominal voltage for TRS 1-8, minimum gain
headroom = 16
print(f"The nominal input at XLR 1-6 is {nom_dBu_XLR1_6} dBu. "
f"The maximum input voltage is {headroom+nom_dBu_XLR1_6} dBu.")
print(f"The nominal input at XLR 7-8 is {nom_dBu_XLR7_8} dBu. "
f"The maximum input voltage is {headroom+nom_dBu_XLR7_8} dBu.")
print(f"The nominal input at TRS 1-8 is {nom_dBu_TRS1_8} dBu. "
f"The maximum input voltage is {headroom+nom_dBu_TRS1_8} dBu.")

The nominal input at XLR 1-6 is -6 dBu. The maximum input voltage is 10 dBu.
The nominal input at XLR 7-8 is 0 dBu. The maximum input voltage is 16 dBu.
The nominal input at TRS 1-8 is 4 dBu. The maximum input voltage is 20 dBu.


Let's define a function with inputs of microphone sensitivity in mV/Pa and SPL required, that compares this data with OctaCapture's specs:

In [15]:
def results(micSens_mV, desiredSPL): #input mic sensitivity in mV/Pa
    micOut_dBu=dBu(10**((desiredSPL-SPL(1))/20)*micSens_mV/1000) #output of the microphone at desired SPL
    if (headroom+nom_dBu_XLR1_6) >= micOut_dBu:
        print (f"You can use XLRs 1-6 to measure {desiredSPL} dB SPL with a {micSens_mV:.1f} mV/Pa microphone. \n"
          f"You can gain it up to {headroom+nom_dBu_XLR1_6-micOut_dBu:.1f} dB above the minimal gain. \n")
    else:
        print (f"You can NOT use XLRs 1-6 to measure {desiredSPL} dB SPL with a {micSens_mV:.1f} mV/Pa microphone. \n"
          f"You would overload it by {abs(headroom+nom_dBu_XLR1_6-micOut_dBu):.1f} dB at this SPL. \n")
    if (headroom+nom_dBu_XLR7_8) >= micOut_dBu:
        print (f"You can use XLRs 7-8 to measure {desiredSPL} dB SPL with a {micSens_mV:.1f} mV/Pa microphone. \n"
          f"You can gain it up to {headroom+nom_dBu_XLR7_8-micOut_dBu:.1f} dB above the minimal gain. ")
    else:
        print (f"You can NOT use XLRs 7-8 to measure {desiredSPL} dB SPL with a {micSens_mV:.1f} mV/Pa microphone. \n"
          f"You would overload it by {abs(headroom+nom_dBu_XLR7_8-micOut_dBu):.1f} dB at this SPL") 
        

## Let's see the results:

#### Let's see the results for a microphone with a 6 mV/Pa sensitivity, like iSemcon EMX7150:

In [21]:
results(6, 140)

You can use XLRs 1-6 to measure 140 dB SPL with a 6.0 mV/Pa microphone. 
You can gain it up to 6.2 dB above the minimal gain. 

You can use XLRs 7-8 to measure 140 dB SPL with a 6.0 mV/Pa microphone. 
You can gain it up to 12.2 dB above the minimal gain. 


#### Let's see the results for a microphone with a 15 mV/Pa sensitivity, like Mellab MYc-3:

In [20]:
results(15, 140)

You can NOT use XLRs 1-6 to measure 140 dB SPL with a 15.0 mV/Pa microphone. 
You would overload it by 1.8 dB at this SPL. 

You can use XLRs 7-8 to measure 140 dB SPL with a 15.0 mV/Pa microphone. 
You can gain it up to 4.2 dB above the minimal gain. 


#### Let's see the results for a microphone with a 34 mV/Pa sensitivity, like Earthworks M23:

In [19]:
results(34, 140)

You can NOT use XLRs 1-6 to measure 140 dB SPL with a 34.0 mV/Pa microphone. 
You would overload it by 8.9 dB at this SPL. 

You can NOT use XLRs 7-8 to measure 140 dB SPL with a 34.0 mV/Pa microphone. 
You would overload it by 2.9 dB at this SPL
